# Pipeline OCR para Notas Fiscais - Demonstração Completa

Este notebook demonstra o pipeline completo de extração de dados de Notas Fiscais brasileiras usando OCR.

## Estrutura do Pipeline

```
Imagem NF → Pré-processamento → OCR → Extração de Campos → Dados Estruturados (JSON)
```

## Sumário

1. [Configuração e Instalação](#1-configuração-e-instalação)
2. [Base de Dados - Justificativa](#2-base-de-dados---justificativa)
3. [Técnicas de Tratamento de Imagens](#3-técnicas-de-tratamento-de-imagens)
4. [Modelos OCR Pré-treinados](#4-modelos-ocr-pré-treinados)
5. [Demonstração do Pipeline Completo](#5-demonstração-do-pipeline-completo)
6. [Uso via API](#6-uso-via-api)

## 1. Configuração e Instalação

### Instalação das dependências

In [ ]:
# Instalação das dependências (executar apenas uma vez)
# !pip install -r ../requirements.txt

In [ ]:
# Imports necessários
import sys
from pathlib import Path

# Adiciona o diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import json

# Imports do projeto
from src.preprocessing import ImageProcessor
from src.ocr import OCREngine, OCRResult
from src.extraction import NFExtractor, NFData
from src.config import PREPROCESSING_CONFIG, OCR_CONFIG, EXTRACTION_CONFIG

print("Imports realizados com sucesso!")

## 2. Base de Dados - Justificativa

### Por que não utilizamos um dataset de treinamento tradicional?

Este projeto utiliza **modelos OCR pré-treinados**, o que elimina a necessidade de um dataset de treinamento próprio. Aqui está a justificativa:

#### Vantagens dos Modelos Pré-treinados:

1. **Treinamento Massivo**: EasyOCR e PaddleOCR foram treinados em milhões de imagens de texto em dezenas de idiomas

2. **Generalização**: Os modelos já aprenderam a reconhecer fontes, tamanhos e estilos variados

3. **Eficiência**: Não requer GPU para treinamento, apenas para inferência

4. **Manutenção**: Atualizações dos modelos são feitas pela comunidade

#### Dados Utilizados:

- **Entrada**: Imagens de Notas Fiscais (DANFE - Documento Auxiliar da NF-e)
- **Formato**: JPG, PNG, TIFF, BMP
- **Características**: Layout padronizado pela legislação brasileira

#### Campos do DANFE (Nota Fiscal Eletrônica):

| Campo | Descrição |
|-------|----------|
| Chave de Acesso | 44 dígitos únicos |
| CNPJ Emitente | Identificação do vendedor |
| CNPJ Destinatário | Identificação do comprador |
| Número NF | Número sequencial |
| Data Emissão | Data da operação |
| Valor Total | Valor da nota |

## 3. Técnicas de Tratamento de Imagens

O pré-processamento é **fundamental** para a qualidade do OCR. Cada técnica tem uma justificativa específica.

In [ ]:
# Inicializa o processador de imagens
processor = ImageProcessor(PREPROCESSING_CONFIG)

# Mostra configurações
print("Configurações de Pré-processamento:")
print(json.dumps(PREPROCESSING_CONFIG, indent=2))

### 3.1 Justificativa das Técnicas

| Técnica | Justificativa |
|---------|---------------|
| **Escala de Cinza** | Reduz complexidade, remove informação de cor irrelevante para texto |
| **Redimensionamento** | Mínimo 300 DPI para boa leitura; muito grande = mais memória |
| **Binarização** | Separa texto do fundo de forma clara para o OCR |
| **Remoção de Ruído** | Artefatos de digitalização prejudicam detecção de caracteres |
| **Correção de Inclinação** | Documentos tortos reduzem precisão do OCR significativamente |
| **CLAHE (Contraste)** | Melhora legibilidade de documentos desbotados ou com iluminação irregular |

In [ ]:
def demonstrate_preprocessing(image_path):
    """
    Demonstra cada etapa do pré-processamento.
    """
    # Carrega imagem
    processor = ImageProcessor(PREPROCESSING_CONFIG)
    
    # Processa com retorno de etapas
    result, steps = processor.process(image_path, return_steps=True)
    
    # Visualiza etapas
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    step_names = [
        ('original', 'Original'),
        ('resized', 'Redimensionada'),
        ('grayscale', 'Escala de Cinza'),
        ('denoised', 'Sem Ruído'),
        ('contrast_enhanced', 'Contraste (CLAHE)'),
        ('deskewed', 'Corrigida (Deskew)'),
    ]
    
    for ax, (key, title) in zip(axes.flat, step_names):
        if key in steps:
            img = steps[key]
            if len(img.shape) == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            ax.imshow(img, cmap='gray' if len(img.shape) == 2 else None)
        ax.set_title(title)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return result

# Para executar com uma imagem real:
# processed = demonstrate_preprocessing('caminho/para/nota_fiscal.jpg')
print("Função demonstrate_preprocessing() pronta para uso.")
print("Uso: processed = demonstrate_preprocessing('caminho/para/imagem.jpg')")

## 4. Modelos OCR Pré-treinados

### Justificativa da Escolha dos Modelos

In [ ]:
# Tabela comparativa dos modelos
modelos_info = """
╔══════════════╦═══════════════════════════════════════════════════════════════════════╗
║ Modelo       ║ Características                                                        ║
╠══════════════╬═══════════════════════════════════════════════════════════════════════╣
║ EasyOCR      ║ • Baseado em Deep Learning (CRAFT + CRNN)                             ║
║ (Principal)  ║ • Excelente suporte a português brasileiro                            ║
║              ║ • Detecta texto em ângulos variados                                   ║
║              ║ • Precisão: 95-98% em texto impresso                                  ║
║              ║ • GPU acceleration disponível                                         ║
╠══════════════╬═══════════════════════════════════════════════════════════════════════╣
║ PaddleOCR    ║ • Estado da arte (PP-OCR v4)                                          ║
║ (Alternativa)║ • Muito rápido e preciso                                              ║
║              ║ • Bom para tabelas e layouts estruturados                             ║
║              ║ • Desenvolvido pela Baidu                                             ║
╠══════════════╬═══════════════════════════════════════════════════════════════════════╣
║ Tesseract    ║ • Engine tradicional, mantido pelo Google                             ║
║ (Fallback)   ║ • Confiável e bem documentado                                         ║
║              ║ • LSTM-based desde v4.0                                               ║
║              ║ • Útil como validação/fallback                                        ║
╚══════════════╩═══════════════════════════════════════════════════════════════════════╝
"""
print(modelos_info)

In [ ]:
# Inicializa o engine OCR
print("Inicializando OCR Engine...")
print("(Isso pode demorar na primeira execução - download dos modelos)\n")

try:
    ocr_engine = OCREngine(OCR_CONFIG)
    print(f"Engines disponíveis: {ocr_engine.get_available_engines()}")
except Exception as e:
    print(f"Aviso: {e}")
    print("Instale as dependências: pip install easyocr paddleocr pytesseract")

### 4.1 Por que EasyOCR como Principal?

**Arquitetura do EasyOCR:**

```
Imagem → CRAFT (Detector) → Regiões de Texto → CRNN (Reconhecedor) → Texto
         ↓                                       ↓
    Character Region                     CNN + BiLSTM + CTC
    Awareness for Text                   (Sequence-to-Sequence)
```

**Vantagens específicas para Notas Fiscais:**

1. Detecta texto em diferentes orientações
2. Suporte nativo a português
3. Bom desempenho em fontes pequenas (comum em NFs)
4. Retorna bounding boxes precisos

## 5. Demonstração do Pipeline Completo

### Pipeline: Imagem → Pré-processamento → OCR → Extração → JSON

In [ ]:
class NFPipeline:
    """
    Pipeline completo de extração de dados de Notas Fiscais.
    
    Etapas:
    1. Pré-processamento da imagem
    2. OCR para extração de texto
    3. Extração de campos estruturados
    4. Validação e formatação
    """
    
    def __init__(self):
        self.processor = ImageProcessor(PREPROCESSING_CONFIG)
        self.ocr = OCREngine(OCR_CONFIG)
        self.extractor = NFExtractor(EXTRACTION_CONFIG)
        
    def process(self, image_path, visualize=True):
        """
        Processa uma imagem de Nota Fiscal.
        
        Args:
            image_path: Caminho para a imagem
            visualize: Se deve mostrar visualizações
            
        Returns:
            dict com dados extraídos
        """
        print("="*60)
        print("PIPELINE OCR - NOTA FISCAL")
        print("="*60)
        
        # 1. Pré-processamento
        print("\n[1/4] Pré-processamento da imagem...")
        processed_image, steps = self.processor.process(
            image_path, 
            return_steps=True
        )
        print(f"   ✓ Imagem processada: {processed_image.shape}")
        
        if visualize:
            self._show_preprocessing(steps)
        
        # 2. OCR
        print("\n[2/4] Executando OCR...")
        ocr_results = self.ocr.extract_text(processed_image, detail=True)
        filtered_results = self.ocr.filter_by_confidence(ocr_results)
        full_text = self.ocr.get_full_text(filtered_results)
        print(f"   ✓ {len(filtered_results)} regiões de texto detectadas")
        
        if visualize:
            self._show_ocr_results(steps['original'], ocr_results)
        
        # 3. Extração de campos
        print("\n[3/4] Extraindo campos da Nota Fiscal...")
        nf_data = self.extractor.extract(full_text)
        print(f"   ✓ {nf_data.campos_extraidos} campos extraídos")
        print(f"   ✓ Confiança: {nf_data.confidence_score:.1%}")
        
        # 4. Resultado final
        print("\n[4/4] Formatando resultado...")
        result = nf_data.to_dict()
        
        print("\n" + "="*60)
        print("DADOS EXTRAÍDOS")
        print("="*60)
        self._print_result(result)
        
        return result
    
    def _show_preprocessing(self, steps):
        """Mostra etapas do pré-processamento."""
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        imgs = [
            ('original', 'Original'),
            ('grayscale', 'Pré-processada'),
            ('deskewed', 'Final'),
        ]
        
        for ax, (key, title) in zip(axes, imgs):
            if key in steps:
                img = steps[key]
                if len(img.shape) == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                ax.imshow(img, cmap='gray' if len(img.shape) == 2 else None)
            ax.set_title(title)
            ax.axis('off')
        
        plt.suptitle('Etapas do Pré-processamento', fontsize=14)
        plt.tight_layout()
        plt.show()
    
    def _show_ocr_results(self, original_image, ocr_results):
        """Mostra resultados do OCR com bounding boxes."""
        img = original_image.copy()
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for result in ocr_results:
            if result.bbox:
                x1, y1, x2, y2 = result.bbox
                color = (0, 255, 0) if result.confidence > 0.7 else (255, 165, 0)
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(img)
        plt.title(f'Detecções OCR ({len(ocr_results)} regiões)')
        plt.axis('off')
        plt.show()
    
    def _print_result(self, result):
        """Imprime resultado formatado."""
        fields = [
            ('numero_nf', 'Número NF'),
            ('serie', 'Série'),
            ('chave_acesso', 'Chave de Acesso'),
            ('data_emissao', 'Data Emissão'),
            ('cnpj_emitente', 'CNPJ Emitente'),
            ('razao_social_emitente', 'Razão Social Emit.'),
            ('cnpj_destinatario', 'CNPJ Destinatário'),
            ('nome_destinatario', 'Nome Destinatário'),
            ('valor_total', 'Valor Total'),
        ]
        
        for key, label in fields:
            value = result.get(key, '')
            if value:
                if key == 'valor_total':
                    print(f"  {label}: R$ {value:,.2f}")
                else:
                    print(f"  {label}: {value}")
        
        print(f"\n  Confiança: {result.get('confidence_score', 0):.1%}")

# Instancia o pipeline
print("Pipeline NFPipeline criado.")
print("\nUso:")
print("  pipeline = NFPipeline()")
print("  resultado = pipeline.process('caminho/para/nota_fiscal.jpg')")

### 5.1 Exemplo de Uso

In [ ]:
# Para testar com uma imagem de Nota Fiscal:

# 1. Coloque uma imagem de NF na pasta samples/
# 2. Execute:

# pipeline = NFPipeline()
# resultado = pipeline.process('../samples/nota_fiscal.jpg')

# O resultado será um dicionário com todos os campos extraídos
print("Para testar:")
print("1. Coloque uma imagem de NF na pasta 'samples/'")
print("2. Execute as células acima")
print("3. Use: pipeline.process('../samples/sua_nota.jpg')")

### 5.2 Teste com Texto Simulado

Para demonstrar a extração de campos sem uma imagem real:

In [ ]:
# Simula texto extraído de uma NF (para demonstração)
texto_simulado = """
DANFE
DOCUMENTO AUXILIAR DA NOTA FISCAL ELETRÔNICA

CHAVE DE ACESSO
3524 0612 3456 7890 1234 5600 0100 0100 0012 3456 7890

NF-e N°: 123456
SÉRIE: 1
DATA DE EMISSÃO: 15/01/2024

EMITENTE
RAZÃO SOCIAL: EMPRESA EXEMPLO LTDA
CNPJ: 12.345.678/0001-90
INSCRIÇÃO ESTADUAL: 123.456.789.000

DESTINATÁRIO
NOME: CLIENTE TESTE S/A
CNPJ: 98.765.432/0001-10

VALOR TOTAL DOS PRODUTOS: R$ 1.234,56
VALOR DO FRETE: R$ 50,00
VALOR TOTAL DA NF: R$ 1.284,56
"""

# Testa extração
extractor = NFExtractor(EXTRACTION_CONFIG)
nf_data = extractor.extract(texto_simulado)

print("RESULTADO DA EXTRAÇÃO (TEXTO SIMULADO)")
print("="*50)
print(json.dumps(nf_data.to_dict(), indent=2, ensure_ascii=False))

## 6. Uso via API

O backend fornece uma API REST para integração com o frontend.

In [ ]:
# Exemplo de chamada à API (após iniciar o servidor)

api_example = """
# Iniciar o servidor:
# python run_api.py

# Endpoints disponíveis:
# - GET  /health         → Status da API
# - POST /ocr            → Apenas OCR (texto bruto)
# - POST /extract        → Extração completa de dados

# Exemplo com curl:
curl -X POST "http://localhost:8000/extract" \
  -H "accept: application/json" \
  -H "Content-Type: multipart/form-data" \
  -F "file=@nota_fiscal.jpg"

# Exemplo com Python requests:
import requests

with open('nota_fiscal.jpg', 'rb') as f:
    response = requests.post(
        'http://localhost:8000/extract',
        files={'file': f}
    )
    
data = response.json()
print(data)
"""

print(api_example)

### 6.1 Documentação da API

Após iniciar o servidor, acesse:
- **Swagger UI**: http://localhost:8000/docs
- **ReDoc**: http://localhost:8000/redoc

In [ ]:
# Estrutura da resposta da API /extract
response_example = {
    "success": True,
    "data": {
        "numero_nf": "123456",
        "serie": "1",
        "chave_acesso": "35240612345678901234560001000100001234567890",
        "data_emissao": "15/01/2024",
        "cnpj_emitente": "12.345.678/0001-90",
        "razao_social_emitente": "EMPRESA EXEMPLO LTDA",
        "cnpj_destinatario": "98.765.432/0001-10",
        "nome_destinatario": "CLIENTE TESTE S/A",
        "valor_total": 1284.56,
        "confidence_score": 0.85,
        "campos_extraidos": 9
    },
    "processing_info": {
        "original_size": [1200, 800],
        "processed_size": [1200, 800],
        "ocr_engine": "easyocr",
        "total_detections": 45,
        "filtered_detections": 38
    }
}

print("Exemplo de resposta da API /extract:")
print(json.dumps(response_example, indent=2))

## Resumo

### Escolhas Técnicas e Justificativas

| Componente | Escolha | Justificativa |
|------------|---------|---------------|
| **Base de Dados** | Modelos pré-treinados | EasyOCR/PaddleOCR já treinados em milhões de imagens |
| **Pré-processamento** | OpenCV | Biblioteca robusta, técnicas específicas para documentos |
| **OCR Principal** | EasyOCR | Deep Learning, suporte a PT-BR, bounding boxes |
| **OCR Alternativo** | PaddleOCR | Estado da arte, rápido, bom para tabelas |
| **Extração** | Regex + Validação | Padrões conhecidos (CNPJ, datas), validação de dígitos |
| **API** | FastAPI | Moderna, rápida, documentação automática |

### Pipeline Completo

```
┌─────────────┐    ┌──────────────────┐    ┌─────────┐    ┌───────────┐    ┌──────────┐
│   Imagem    │ → │ Pré-processamento │ → │   OCR   │ → │ Extração  │ → │   JSON   │
│   (DANFE)   │    │  (OpenCV)        │    │(EasyOCR)│    │  (Regex)  │    │ (Dados)  │
└─────────────┘    └──────────────────┘    └─────────┘    └───────────┘    └──────────┘
```